# **LP1_Run1**

In [ ]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [ ]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))


### **Generate Mathematical Model**

In [ ]:
problem = """A firm from Milan sells chemical products for professional cosmetics. It is planning the production of three products, GCA, GCB and GCC, for a given period of
time by mixing two different components: C1 and C2. All the end products must
contain at least one of the two components, and not necessarily both.

For the next planning period, 10,000 l of C1 and 15,000 l of C2 are available.
The production of GCA, GCB and GCC must be scheduled to at least cover the
minimum demand level of 6,000, 7,000 and 9,000 l, respectively. It is assumed
that when chemical components are mixed, there is no loss or gain in volume.

Each chemical component, C1 and C2, has a proportional critical element, 0.4
and 0.2, respectively. That is to say, each litre of C1 contains 0.4 l of the critical
element. To obtain GCA, the mixture must proportionally contain at least a 0.3
fraction of the critical element. Another requirement is that the quantity of the
critical element is seen in GCB, a 0.3 fraction at the most.
Furthermore, the minimum ratio of C1 with C2 in product GCC must be 0.3.

The profit expected for the sale of each litre of GCA, GCB and GCC is $120, $135
and $155, respectively.

Optimise the production planning of this firm.  """

In [ ]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [ ]:
print(response1.choices[0].message.content)

Parameters:

- Availability of component C1: A1 = 10,000 L
- Availability of component C2: A2 = 15,000 L
- Minimum demand for products GCA: D1 = 6,000 L
- Minimum demand for products GCB: D2 = 7,000 L
- Minimum demand for products GCC: D3 = 9,000 L
- Critical element in component C1: E1 = 0.4
- Critical element in component C2: E2 = 0.2
- Profit for the sale of each litre of GCA: P1 = $120
- Profit for the sale of each litre of GCB: P2 = $135
- Profit for the sale of each litre of GCC: P3 = $155

Decision Variables:

Let X1, X2 and X3 be the volume of GCA, GCB and GCC to be produced, respectively. 

Let Y1 and Y2 be the volume of component C1 used in all products, and Y3 and Y4 be the volume of component C2 used in all products.

Objective Function:

Maximize Profit = P1 * X1 + P2 * X2 + P3 * X3

Constraints:

Material availability constraints:

- Y1 + Y2 <= A1 (Availability of C1)
- Y3 + Y4 <= A2 (Availability of C2)

Product ratio constraint for GCC:
- Y2/Y4 >= 0.3  

Demand constrai

### **Generate Pyomo Code**

In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
Markdown(response2.choices[0].message.content)

Here's the Python code using pyomo:

```python
from pyomo.environ import *

model = ConcreteModel()

# Define Sets
PRODUCTS = ['GCA', 'GCB', 'GCC']

# Define Parameters
A1 = 10000
A2 = 15000
D = {'GCA': 6000, 'GCB': 7000, 'GCC': 9000}
E = {'C1': 0.4, 'C2': 0.2}
P = {'GCA': 120, 'GCB': 135, 'GCC': 155}

# Define Variables
model.y = Var(PRODUCTS, within=NonNegativeReals)
model.x = Var(PRODUCTS, within=NonNegativeReals)

# Define Objective
model.profit = Objective(expr=sum(P[i]*model.x[i] for i in PRODUCTS), 
                         sense=maximize)

# Define Constraints
model.con_material_C1 = Constraint(expr=model.y['GCA'] + model.y['GCC'] <= A1)
model.con_material_C2 = Constraint(expr=model.y['GCB'] + model.y['GCC'] <= A2)
model.con_product_ratio = Constraint(expr=model.y['GCC'] / model.y['GCB'] >= 0.3)

for i in PRODUCTS:
    model.add_component("con_demand_"+str(i), Constraint(expr=model.x[i] >= D[i]))
    model.add_component("con_produce_"+str(i), Constraint(
        expr=model.x[i] == model.y[i]))

model.con_element_GCA = Constraint(expr=E['C1']*model.y['GCA'] + E['C2']*model.y['GCC'] >= 0.3* model.x['GCA'])
model.con_element_GCB = Constraint(expr=E['C1']*model.y['GCC'] + E['C2']*model.y['GCB'] <= 0.3* model.x['GCB'])

# solve
SolverFactory('gurobi').solve(model)

model.pprint()
```
In this code, data such as the availabilities, demands, critical element percentages, and profits are represented as parameters that you can tweak as necessary. Our objective is to maximize profit under given constraints. Constraints include the availability of components, demand of products, product ratios, product production constraints, and critical element constraints in products. Finally, an optimization solver is used to solve the given constraints and objective function. This code uses the gurobi solver, but this can be replaced with the solver of your choice.

Note: The last pprint() line prints out the final model showing the constraints used, variables along with their optimal values, reused constraints, and the optimal objective function value.

### **Run the code Generated by GPT4**

In [ ]:
from pyomo.environ import *

model = ConcreteModel()

# Define Sets
PRODUCTS = ['GCA', 'GCB', 'GCC']

# Define Parameters
A1 = 10000
A2 = 15000
D = {'GCA': 6000, 'GCB': 7000, 'GCC': 9000}
E = {'C1': 0.4, 'C2': 0.2}
P = {'GCA': 120, 'GCB': 135, 'GCC': 155}

# Define Variables
model.y = Var(PRODUCTS, within=NonNegativeReals)
model.x = Var(PRODUCTS, within=NonNegativeReals)

# Define Objective
model.profit = Objective(expr=sum(P[i]*model.x[i] for i in PRODUCTS),
                         sense=maximize)

# Define Constraints
model.con_material_C1 = Constraint(expr=model.y['GCA'] + model.y['GCC'] <= A1)
model.con_material_C2 = Constraint(expr=model.y['GCB'] + model.y['GCC'] <= A2)
model.con_product_ratio = Constraint(expr=model.y['GCC'] / model.y['GCB'] >= 0.3)

for i in PRODUCTS:
    model.add_component("con_demand_"+str(i), Constraint(expr=model.x[i] >= D[i]))
    model.add_component("con_produce_"+str(i), Constraint(
        expr=model.x[i] == model.y[i]))

model.con_element_GCA = Constraint(expr=E['C1']*model.y['GCA'] + E['C2']*model.y['GCC'] >= 0.3* model.x['GCA'])
model.con_element_GCB = Constraint(expr=E['C1']*model.y['GCC'] + E['C2']*model.y['GCB'] <= 0.3* model.x['GCB'])

# solve
SolverFactory('glpk').solve(model)

model.pprint()

ValueError: Model constraint (con_product_ratio) contains nonlinear terms that cannot be written to LP format

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [ ]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, NonNegativeReals, SolverFactory

# Create a model
model = ConcreteModel()

# Decision Variables
model.X1 = Var(domain=NonNegativeReals)
model.X2 = Var(domain=NonNegativeReals)
model.X3 = Var(domain=NonNegativeReals)
model.Y1 = Var(domain=NonNegativeReals)
model.Y2 = Var(domain=NonNegativeReals)
model.Y3 = Var(domain=NonNegativeReals)
model.Y4 = Var(domain=NonNegativeReals)

# Parameters
A1 = 10000
A2 = 15000
D1 = 6000
D2 = 7000
D3 = 9000
E1 = 0.4
E2 = 0.2
P1 = 120
P2 = 135
P3 = 155

# Objective Function
model.Profit = Objective(expr=P1*model.X1 + P2*model.X2 + P3*model.X3, sense=maximize)

# Constraints
model.C1_Availability = Constraint(expr=model.Y1 + model.Y2 <= A1)
model.C2_Availability = Constraint(expr=model.Y3 + model.Y4 <= A2)
model.GCC_Ratio = Constraint(expr=model.Y2 >= 0.3*model.Y4)
model.GCA_Demand = Constraint(expr=model.X1 >= D1)
model.GCB_Demand = Constraint(expr=model.X2 >= D2)
model.GCC_Demand = Constraint(expr=model.X3 >= D3)
model.GCA_Production = Constraint(expr=model.X1 == model.Y1 + model.Y3)
model.GCB_Production = Constraint(expr=model.X2 == model.Y2 + model.Y4)
model.GCC_Production = Constraint(expr=model.X3 == model.Y2 + model.Y4)
model.GCA_Critical = Constraint(expr=E1*model.Y1 + E2*model.Y3 >= 0.3 * model.X1)
model.GCB_Critical = Constraint(expr=E1*model.Y2 + E2*model.Y4 <= 0.3 * model.X2)

# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model)

# Display the results
model.display()


Model unknown

  Variables:
    X1 : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 6000.0 :  None : False : False : NonNegativeReals
    X2 : Size=1, Index=None
        Key  : Lower : Value   : Upper : Fixed : Stale : Domain
        None :     0 : 19000.0 :  None : False : False : NonNegativeReals
    X3 : Size=1, Index=None
        Key  : Lower : Value   : Upper : Fixed : Stale : Domain
        None :     0 : 19000.0 :  None : False : False : NonNegativeReals
    Y1 : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 5615.38461538462 :  None : False : False : NonNegativeReals
    Y2 : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 4384.61538461538 :  None : False : False : NonNegativeReals
    Y3 : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
       